In [1]:
from mako.template import Template
from datetime import datetime
import os
import netCDF4 as nc

In [ ]:
def results_dataset(period, grid, results_dir):
    """Return the results dataset for period (e.g. 1h or 1d)
    and grid (e.g. grid_T, grid_U) from results_dir.
    """
    filename_pattern = 'SalishSea_{period}_*_{grid}.nc'
    filepaths = glob(os.path.join(results_dir, filename_pattern.format(period=period, grid=grid)))
    return nc.Dataset(filepaths[0])

In [ ]:
#model_path = '/ocean/sallen/allen/research/MEOPAR/Operational/'
results_home = '/ocean/nsoontie/MEOPAR/sprint/'
grid_T_hr = results_dataset('1h', 'grid_T', results_home)

for name in names:
    
    # Get sea surface height
    lat = SITES[name]['lat']
    lon = SITES[name]['lon']
    j, i = tidetools.find_closest_model_point(lon, lat, X, Y, bathy, allow_land=False)
    ssh_loc = grid_T.variables['sossheig'][:, j, i]

    # Get tides and ssh
    ttide = get_tides(name)
    ssh_corr = correct_model_ssh(ssh_loc, t, ttide)
    
    # Information
    res = compute_residual(ssh_loc, t, ttide)
    [max_ssh, index_ssh, tmax, max_res, max_wind, ind_w] = get_maxes(
        ssh_corr, t, res, lon, lat, model_path)
    max_sshs[name] = max_ssh
    max_times[name] = tmax
    max_winds[name] = max_wind

In [2]:
def make_web_strings(Stations):
    '''Construct strings for surge warning zones and times of day and return them as part of the input dict STATIONS
    '''
    
    # Initialize WARNINGS string
    Warnings = ''
    Threshold = 1

    # Iterate through tide stations
    for station in Stations:
        
        # Add areas to WARNINGS string
        if Stations[station]['max_sealevel'] > threshold:
            Warnings = Warnings + Stations[station]['area'] + ', '
        
        # Define times of day in readable format (e.g. 25-Aug-2015 06:00 becomes "early Tuesday morning")
        # and append to STATIONS dict
        Stations[station]['day'] = Stations[station]['date'].strftime('%A')
        if Stations[station]['date'].hour < 12:
            Stations[station]['time'] = 'morning'
            if Stations[station]['date'].hour < 8:
                Stations[station]['period'] = 'early'
            else:
                Stations[station]['period'] = 'late'
        elif Stations[station]['date'].hour >= 12 and Stations[station]['date'].hour < 17:
            Stations[station]['time'] = 'afternoon'
            if Stations[station]['date'].hour < 15:
                Stations[station]['period'] = 'early'
            else:
                Stations[station]['period'] = 'late'
        else:
            Stations[station]['time'] = 'evening'
            if Stations[station]['date'].hour < 20:
                Stations[station]['period'] = 'early'
            else:
                Stations[station]['period'] = 'late'
    
    # Final WARNINGS syntax and append to STATIONS dict
    Stations['Warnings'] = Warnings[:-2][::-1].replace(',', ';', 1)[::-1].replace(';', ' and')
    
    return Stations

In [3]:
# STATIONS dict template
Stations = {'PA': {'name': 'Point Atkinson', 'area': 'Vancouver',      'max_sealevel': 1.5, 'windspeed': 2.5, 'date': datetime.now()},
            'VI': {'name': 'Victoria',       'area': 'Victoria',       'max_sealevel': 0.3, 'windspeed': 2.3, 'date': datetime.now()},
            'CP': {'name': 'Cherry Point',   'area': 'Boundary Bay',   'max_sealevel': 0.2, 'windspeed': 1.5, 'date': datetime.now()},
            'CR': {'name': 'Campbell River', 'area': 'Campbell River', 'max_sealevel': 0.1, 'windspeed': 0.5, 'date': datetime.now()}}

# Make WARNINGS and readable time strings
Stations = make_web_strings(Stations)

# Generate RST in Mako
print(Template(filename='www/templates/surgetext.mako').render(**Stations))

    STORM SURGE ADVISORY! Extreme sea levels expected for the marine areas of **Vancouver**

Synopsis:
Strong winds over the northeast Pacific Ocean are expected to produce elevated sea levels near Vancouver late Tuesday afternoon. These elevated sea levels may present a flood risk to coastal structures and communities at high tide.

Point Atkinson
Maximum Water Level: 1.5 m
Wind speed: 2.5 m/s
Time: Aug 25, 2015 16:41 [PST]

Victoria
Maximum Water Level: 0.3 m
Wind speed: 2.3 m/s
Time: Aug 25, 2015 16:41 [PST]

Cherry Point
Maximum Water Level: 0.2 m
Wind speed: 1.5 m/s
Time: Aug 25, 2015 16:41 [PST]

Campbell River
Maximum Water Level: 0.1 m
Wind speed: 0.5 m/s
Time: Aug 25, 2015 16:41 [PST]
